# Load data

In [1]:
import pandas as pd
import numpy as np
import scipy as sy
import os
from scipy import stats
from more_itertools import numeric_range
import math
import seaborn as sb

In [2]:
# In our setup we monitor the start of the miniscope recording session via the light on the miniscope, which is recorded with a behavior camera.
# We determine determine the time when the behavior session begins and the time when the miniscope begins recording. We substract the timing in the next cell. 

In [3]:
# Specify the time in sec to be deleted from analysis. 
bf_time=[4.3,6.5,6.4,5.5,5.1,2.3,4.6,5.3,14.1,6.0,5.4,4.4,5.0,14.0,5.7,5.4,7.9,6.5]
t=0

In [4]:
# upload file
os.listdir('C:/Users/allenli/caiman_data/github codes')

['.ipynb_checkpoints',
 '153_Day11_Standardizing data.ipynb',
 '153_Day11_SUC_concat-updating spatial components.csv',
 'desktop.ini',
 'Standardizing data and time diff deleted.ipynb']

In [6]:
files=os.listdir('C:/Users/allenli/caiman_data/github codes')[1]

In [7]:
name = files[:-10]
name

'153_Day11_SUC_concat-updating spatial comp'

In [8]:
#Delta f over f is one type of standardization 

In [9]:
### Delta f over f
t=0
for files in os.listdir('C:/Users/allenli/caiman_data/github codes'):
    print(files)
    print(bf_time[t])

    raw_data = pd.read_csv('C:/Users/allenli/caiman_data/github codes'+files)
    name = files[:-10]
    before_time = bf_time[t]
    t=t+1
    data = raw_data.copy()
    data = data - min(data.iloc[:,range(1,len(data.columns))].min())
    data['ceil_time'] = (data['Time(ms)']/100).apply(np.ceil).astype(int)*100
    data = data.groupby(['ceil_time']).mean()
    data = data.reset_index()
    data['Time(ms)'] = data['ceil_time']
    data = data.drop('ceil_time', axis=1)
    data = data.rename(columns={'Time(ms)':'Time(s)'})
    data['Time(s)'] = round(data['Time(s)']/1000,1)
    delete_indx = data[data['Time(s)'] <= before_time].index
    #print(delete_indx)
    data.drop(delete_indx, inplace=True)
    data.reset_index(drop=True, inplace=True)
    data['Time(s)'] = data['Time(s)'] - before_time
    #data.head()
    cols = data.columns
    z_data = pd.DataFrame(columns=cols)
    z_data['Time(s)'] = data['Time(s)']
    for xc in range(1,len(cols)):
        for ran in range(0,math.ceil(len(data)/3000)):
            if (ran+1)*3000<len(data):
                baseline = data.iloc[range(ran*3000,(ran+1)*3000),xc].mean()
                z_data.iloc[range(ran*3000,(ran+1)*3000),xc] = (data.iloc[range(ran*3000,(ran+1)*3000),xc] - baseline)/baseline
            else:
                baseline = data.iloc[range(len(data)-3000,len(data)),xc].mean()
                z_data.iloc[range(ran*3000,len(data)),xc] = (data.iloc[range(ran*3000,len(data)),xc] - baseline)/baseline

    z_data["Time(s)"]=round(z_data["Time(s)"],1)
    z_data.to_csv('C:/Users/allenli/caiman_data/github codes'+name +'_deltaF_over_F.csv', index=False)

.ipynb_checkpoints
4.3


FileNotFoundError: [Errno 2] File C:/Users/allenli/caiman_data/github codes.ipynb_checkpoints does not exist: 'C:/Users/allenli/caiman_data/github codes.ipynb_checkpoints'

In [ ]:
#Z-score is another type of standardization 

In [12]:

### z-score
for files in os.listdir('C:/Users/allenli/caiman_data/github codes'):
    print(files)
    print(bf_time[t])

    raw_data = pd.read_csv('C:/Users/allenli/caiman_data/github codes'+files)
    name = files[:-4]
    before_time = bf_time[t]
    t=t+1
    data = raw_data.copy()
    data['ceil_time'] = (data['Time(ms)']/100).apply(np.ceil).astype(int)*100
    data = data.groupby(['ceil_time']).mean()
    data = data.reset_index()
    data['Time(ms)'] = data['ceil_time']
    data = data.drop('ceil_time', axis=1)
    data = data.rename(columns={'Time(ms)':'Time(s)'})
    data['Time(s)'] = round(data['Time(s)']/1000,1)
    delete_indx = data[data['Time(s)'] <= before_time].index
    #print(delete_indx)
    data.drop(delete_indx, inplace=True)
    data.reset_index(drop=True, inplace=True)
    data['Time(s)'] = data['Time(s)'] - before_time
    #data.head()
    cols = raw_data.columns
    cols = cols[1:]
    z_data = pd.DataFrame()
    for col in cols:
        col_zscore = col + '_zscore'
        z_data['Time(s)'] = data['Time(s)']
        z_data[col] = (data[col] - data[col].mean())/data[col].std(ddof=0)

    z_data["Time(s)"]=round(z_data["Time(s)"],1)
    #z_data.tail()
    z_data.to_csv('C:/Users/allenli/caiman_data/github codes'+name +'_standardized.csv', index=False)

D1#60_WD2.csv
5.4
D1#60_WD21.csv
4.9
D1#62_WD2.csv
2.4
D1#62_WD21.csv
3.4
